# Creating a Chatbot with Upstage Solar API Endpoint, Langchain and Gradio

This lab goes through the creation of a chatbot with the following details :
- Uses the Solar LLM API endpoint from Upstage.
- Uses Langchain for simplifying the API calls to Upstage Solar API and to also maintain context (chat history)
- Uses Gradio for the simple chatbot UI

### 1. Install the python modules

In [1]:
!pip install --upgrade -q langchain langchain-community langchainhub langchain-openai llama-index


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install --upgrade -q gradio


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. Get the Solar API Endpoint key and setup the base url

Note that you will need to signup for the Solar API endpoint. It is in a free beta until the end of March 2024. From there, you can get your API key.

In [3]:
from getpass import getpass

BASE_URL = 'https://api.upstage.ai/v1/solar'
APIKEY = getpass()

### 3. Let's test out the Solar API with a simple chat API call

In [4]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat_model = ChatOpenAI(base_url=BASE_URL, model_name="solar-1-mini-chat", api_key=APIKEY)
messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content="Hi, how are you?"
    )
]

response = chat_model.invoke(messages)
print(response)

content="Hello! I'm just a computer program, so I don't have feelings or emotions, but I'm here to help you with any questions or problems you may have. How can I assist you today?"


### 4. And here is the code for the full chatbot

- I think this code is pretty self explanatory.
- Note that it won't work necessarily in a cloud hosted notebook (like on Azure or maybe Google Colab). I don't think so anyway. The gradio chatbot assumes localhost. Personally, I did this with vscode and it worked quite well. Gardio runs very well on vscode.

In [24]:
import gradio as gr
import random
import time
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_model = ChatOpenAI(base_url=BASE_URL, model_name="solar-1-mini-chat", api_key=APIKEY)
chat_history = ChatMessageHistory()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat_model

def doResponse(user_prompt):
    global chat_model, chat_history

    chat_history.add_user_message(("(act as human friend to user. "
                                   "Don't act like an ai. "
                                   "Don't act like an assistant. " 
                                   "Do not break character) user prompt: ") + user_prompt)

    response = chain.invoke({"messages": chat_history.messages})

    chat_history.add_ai_message(response)
    return response.content

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, ch):
        time.sleep(2)
        bot_message = doResponse(message)
        ch.append((message, bot_message))
        return "", ch

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


h1
h2
h3
content='Hello! How can I help you today?'
h1
h2
h3
content="The sky can be many different colors depending on the time of day and the weather. During the day, it's usually a bright blue color, but at sunrise and sunset, it can turn pink or orange. At night, it's usually dark, but you might be able to see stars if it's clear out."
h1
h2
h3
content='Yes, I\'m familiar with the poem "The Walrus and the Carpenter" by Lewis Carroll. It\'s a silly and whimsical poem from the same author who wrote "Alice in Wonderland." In the poem, the walrus and the carpenter invite a group of oysters to a feast, but it turns out that the walrus and the carpenter are planning to eat the oysters themselves! It\'s a fun poem with a twist ending.'
